In [1]:
import json

from py_constants import CODE_INPUT, XML_OUTPUT, FUNCTION_MAP, JSON_FILE
%run py_common.py ## Common Methods
%run py_feature_extraction.ipynb ##Feature Extraction

#### Extracting Student Features. (Only those features are checked which are present in teachers solution)

In [2]:
def extract_features(root, json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    features = {}
    for key, function_name in FUNCTION_MAP.items():
        if key in data:
            function = globals()[function_name]
            features[key] = function(root)
    
    return features, data

In [3]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    total_score = 0
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Penalize if the student's count exceeds the teacher's count
            if student_count > teacher_count:
                feature_score *= 0.95  # Example penalty factor (adjust as needed)
            # Add the feature score to the total score
            total_score += feature_score
            # Store the individual score for the feature
            individual_scores[feature] = {
                "student_count": student_count,
                "teacher_count": teacher_count,
                "score": f"{student_count}/{teacher_count}",
                "feature_score": feature_score
            }
        else:
            # If the feature is not present in the student's solution
            individual_scores[feature] = {
                "student_count": 0,
                "teacher_count": teacher_count,
                "score": f"0/{teacher_count}",
                "feature_score": 0
            }
    
    # Calculate the final score percentage
    num_features = len(teacher_solution)
    final_score_percentage = total_score / num_features * 100 if num_features > 0 else 0
    return individual_scores, final_score_percentage

# Example usage:
# teacher_solution = {'NF': 4, 'NP': 4, 'NPt-s': 1, 'NAdd-s': 1, 'NAdd-B': 1, 'NAdd-A': 1}
# student_solution = {'NF': 4, 'NP': 7, 'NPt-s': 1, 'NAdd-s': 1, 'NAdd-B': 1, 'NAdd-A': 1}

# individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
# print("Individual scores:", individual_scores)
# print("Final score percentage:", final_score_percentage)


## Main Function

In [4]:
if __name__ == "__main__":
    # Specify the source code file and output file for the AST
    #source_code_file = 'test.cpp'
    #xml_output_file = 'ast.xml'

    # Generate AST
    #generate_ast(source_code_file, xml_output_file)

    # Example usage
    xml_file = CODE_INPUT+"/ast1.xml"
    #xml_file = CODE_INPUT+"/pointers.xml"
    #xml_file = CODE_INPUT+"/arrPointer.xml"
    #xml_file = CODE_INPUT+"/structExample.xml"
    #xml_file = CODE_INPUT+"/typedefExample.xml"
    #xml_file = CODE_INPUT+"/recursionExample.xml"
    #xml_file = CODE_INPUT+"/mallocExample.xml"
    #xml_file = CODE_INPUT+"/sizeofExample.xml"

    root =  parse_xml(xml_file)
    
    json_file = JSON_FILE
    student_solution, teacher_solution = extract_features(root, json_file)

    print("Extracted features from Student:")
    print(student_solution)

    individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
    print("Individual scores:", individual_scores)
    print("Final score percentage:", final_score_percentage)

Extracted features from Student:
{'NF': 3, 'NP': 3, 'NC': 3, 'NR': 6, 'NFRec': 0, 'NCRec': 0, 'NIFPar': 0, 'NRRec': 0, 'NRNRec': 0}
Individual scores: {'NF': {'student_count': 3, 'teacher_count': 3, 'score': '3/3', 'feature_score': 1}, 'NP': {'student_count': 3, 'teacher_count': 3, 'score': '3/3', 'feature_score': 1}, 'NC': {'student_count': 3, 'teacher_count': 4, 'score': '3/4', 'feature_score': 0.75}, 'NR': {'student_count': 6, 'teacher_count': 6, 'score': '6/6', 'feature_score': 1}, 'NFRec': {'student_count': 0, 'teacher_count': 1, 'score': '0/1', 'feature_score': 0}, 'NCRec': {'student_count': 0, 'teacher_count': 1, 'score': '0/1', 'feature_score': 0}, 'NIFPar': {'student_count': 0, 'teacher_count': 1, 'score': '0/1', 'feature_score': 0}, 'NRRec': {'student_count': 0, 'teacher_count': 1, 'score': '0/1', 'feature_score': 0}, 'NRNRec': {'student_count': 0, 'teacher_count': 1, 'score': '0/1', 'feature_score': 0}}
Final score percentage: 41.66666666666667
